In [1]:
from importlib import reload
from model_trainer import ModelTrainer
from train import download_and_unpack_dataset, train

import json

In [2]:
root_dir = "/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/custom_cnn"

config_file = open(root_dir + "/trainer_config.json", 'r')
config = json.load(config_file)
config_file.close()

print(config)

{'model_name': 'Plain18Layer', 'store_model_dir_rel_path': '/savedmodels', 'train_dataset_dir_rel_path': '/data/cifar10/train', 'test_dataset_dir_rel_path': '/data/cifar10/test', 'dataset_root_dir_rel_path': '/data', 'random_seed': 420, 'batch_size': 256, 'epochs': 20, 'learning_rate': 0.01}


In [3]:
download_and_unpack_dataset(root_dir + config['dataset_root_dir_rel_path'])

Data already downloaded


In [ ]:
train(root_dir, config)

Load train data from /home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/custom_cnn/data/cifar10/train
Load test data from /home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/custom_cnn/data/cifar10/test
Detected device:  cpu
Epoce: 0


## Load model and test

In [5]:
from model_plain_18_layer import Plain18Layer
from model_wrapper import ModelWrapper
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torch.utils.data.dataloader import DataLoader

In [6]:
from torchvision.datasets import ImageFolder

test_dataset = ImageFolder(
    root="/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/custom_cnn/data/cifar10/test",
    transform=ToTensor()
)
# Create DataLoader
test_data_loader = DataLoader(
    dataset=test_dataset,
    batch_size=128,
    shuffle=True,
    num_workers=2,
    pin_memory=False  # should be True if cuda is available
)

In [8]:

model = Plain18Layer()
model_wrapper = ModelWrapper(model)

model_wrapper.load_model_weights("/home/erik/Desktop/WiSe2023_24/xAI-Proj-M/xAI-Proj-M/custom_cnn/savedmodels/Plain18Layer_20231203_2001/model_state_dict")

model_wrapper.evaluate_model(test_data_loader)


{'mean_loss': 2.302595615386963, 'mean_acc': 0.10166139155626297}